In [1]:
import wandb

In [2]:
import wandb
import torch
import os, sys
import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar
# from pytorch_lightning.loggers import WandbLogger
import ray
from ray import tune
# from ray.air.integrations.wandb import setup_wandb
from ray.tune.integration.wandb import wandb_mixin
import getpass

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fpl_engineering.data.lit_data_module import FPLDataModule
from fpl_engineering.models.lit_models import FPLLSTMRegressor

from fpl_engineering.utils import get_project_root
project_root = str(get_project_root())
log_dir = project_root+'/logs'

devicpe = 'cuda' if torch.cuda.is_available() else 'cpu'



static_config = {'n_features': 35, 'epochs':40}


In [2]:
import torch.nn as nn

nn.LSTM??

Init signature: nn.LSTM(*args, **kwargs)
Source:        
class LSTM(RNNBase):
    r"""Applies a multi-layer long short-term memory (LSTM) RNN to an input
    sequence.


    For each element in the input sequence, each layer computes the following
    function:

    .. math::
        \begin{array}{ll} \\
            i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi}) \\
            f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}) \\
            g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg}) \\
            o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}) \\
            c_t = f_t \odot c_{t-1} + i_t \odot g_t \\
            h_t = o_t \odot \tanh(c_t) \\
        \end{array}

    where :math:`h_t` is the hidden state at time `t`, :math:`c_t` is the cell
    state at time `t`, :math:`x_t` is the input at time `t`, :math:`h_{t-1}`
    is the hidden state of the layer at time `t-1` or the initial hidden
    state at time `0`, and :math:`i_t`, :math:`f

In [4]:

@wandb_mixin
def sweep_iteration():

    # set up W&B logger
    wandb.init(config=tr_config)    # required to have access to `wandb.config`
    config = wandb.config

    # setup data
    data_module = FPLDataModule(batch_size=config.batch_size, seq_len=config.seq_len, download_data=False)


    # setup model - note how we refer to sweep parameters with wandb.config
    model = FPLLSTMRegressor(n_features= static_config['n_features'], hidden_size=config.hidden_size, seq_len=config.seq_len, 
                            batch_size=config.batch_size, num_layers=config.n_layers, dropout=config.dropout,learning_rate=config.lr)


    # early_stopping_callback = EarlyStopping(monitor='val_loss', patience=4, verbose=True)
    progress_bar = TQDMProgressBar(refresh_rate=1)

    logger = [WandbLoggerCallback]
    callbacks = [progress_bar]

    # setup Trainer
    trainer = pl.Trainer(accelerator = device,
                        max_epochs = static_config['epochs'],
                        logger= logger,
                        callbacks = callbacks,
                        log_every_n_steps=20,
                        precision=16)

    # train
    trainer.fit(model, datamodule=data_module)



api_key = '0e7613d70774bd853ddb2dc316968c77437977be'
ray.init()
tr_config = {  

    "dropout":tune.grid_search([0.3, 0.4, 0.5, 0.75, 0.85]),
    "hidden_size":tune.grid_search([64, 128, 256, 512]),
    "batch_size":tune.grid_search([64, 128, 256, 512]),
    "lr":tune.uniform(0.000001, 0.1),
    "seq_len":tune.grid_search([4,5]),
    "n_layers":tune.grid_search([1,2]),
    "wandb": {
        "project": "FPL",
        "api_key":api_key,
        }

    }
    
analysis = tune.run(
        sweep_iteration,
        config = tr_config,
    )
print(analysis)

2022-11-12 13:42:25,597	ERROR services.py:1403 -- Failed to start the dashboard: Failed to start the dashboard, return code 1
Failed to read dashboard log: [Errno 2] No such file or directory: '/tmp/ray/session_2022-11-12_13-42-24_431391_10809/logs/dashboard.log'
2022-11-12 13:42:25,599	ERROR services.py:1404 -- Failed to start the dashboard, return code 1
Failed to read dashboard log: [Errno 2] No such file or directory: '/tmp/ray/session_2022-11-12_13-42-24_431391_10809/logs/dashboard.log'
Traceback (most recent call last):
  File "/Users/bernie/miniforge3/envs/fpl_engineering/lib/python3.10/site-packages/ray/_private/services.py", line 1375, in start_api_server
    with open(dashboard_log, "rb") as f:
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/ray/session_2022-11-12_13-42-24_431391_10809/logs/dashboard.log'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bernie/miniforge3/envs/fpl_engineering/

: 

: 